In [2]:
import pandas as pd

In [1]:
# El mpetodo de VI intenta resolver el problema de la variable omitida, dejando a la variable inobservable en el término de 
# error, pero en lugar de estimar por MCO, reconoce la presencia de la variable omitida

![](1.png)

In [2]:
# Esa ecuación por MCO , tiene en el residuo la habilidad ( inobservable ) y por ello elcoeficiente de educ es sesgado
# si educ y habilidad estan correlacionadas ( que es lo que se supone)

In [ ]:
# Partiendo de eso se debe encontrar una VI , z, para educ, que aporte nueva información
# Debe poser dos propiedades
# 1) z no correlacionada con u . Exogeneidad del instrumento. Sin efecto en y luego de controlar por educ y omitidas. (Supuesto)
# 2) z correlacionada con educ. Relevancia del instrumento. Correlacion con la endógena, educ. 
#     Relevante para explicar educ(Verificable) 

In [ ]:
# Cuando una x es exógena , se puede utilizar como su propio instrumento 

## Ejemplo 15.1 Estimación del rendimiento de la educación para las mujeres casadas (mroz.dta)

In [18]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS

In [22]:
dfej15_1 = pd.read_stata('mroz.dta')
dfej15_1.head(3)

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310.0,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800.0,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040.0,0.6915,12,7,5.0,0,15,12.039910,1.514138,225


In [9]:
df2 = df1.drop(df1[df1.inlf == 0 ].index)
df2.head(3)

In [10]:
df2.shape

(428, 22)

In [11]:
reg1= smf.ols('lwage ~ educ',data=df2).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     56.93
Date:                Sun, 13 Oct 2019   Prob (F-statistic):           2.76e-13
Time:                        10:16:55   Log-Likelihood:                -441.26
No. Observations:                 428   AIC:                             886.5
Df Residuals:                     426   BIC:                             894.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1852      0.185     -1.000      0.3

In [13]:
reg2= smf.ols('educ ~ fatheduc',data=df2).fit()
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     88.84
Date:                Sun, 13 Oct 2019   Prob (F-statistic):           2.76e-19
Time:                        10:17:22   Log-Likelihood:                -920.02
No. Observations:                 428   AIC:                             1844.
Df Residuals:                     426   BIC:                             1852.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.2371      0.276     37.099      0.0

In [39]:
from statsmodels.sandbox.regression.gmm import IV2SLS 
reg3= IV2SLS(df2.lwage ,df2.educ,instrument=df2.fatheduc).fit()
print(reg3.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                  lwage   R-squared:                       0.615
Model:                         IV2SLS   Adj. R-squared:                  0.614
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Sun, 13 Oct 2019                                         
Time:                        13:55:23                                         
No. Observations:                 428                                         
Df Residuals:                     427                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
educ           0.1360      0.003     39.461      0.0

c:\python\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
c:\python\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
c:\python\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [3]:
from linearmodels.iv import IV2SLS

In [31]:
reg4 = IV2SLS.from_formula(' lwage ~ 1 +  [educ ~ fatheduc] ', data=df2).fit()
print(reg4)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.0934
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0913
No. Observations:                 428   F-statistic:                    2.5656
Date:                Sun, Oct 13 2019   P-value (F-stat)                0.1092
Time:                        13:31:57   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.4411     0.4643     0.9501     0.3421     -0.4689      1.3511
educ           0.0592     0.0369     1.6017     0.10

In [37]:
reg5 = IV2SLS(df2.lwage,np.ones(len(df2)) ,df2.educ, df2.fatheduc).fit(cov_type='unadjusted')
print(reg5)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.0934
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0913
No. Observations:                 428   F-statistic:                    2.8487
Date:                Sun, Oct 13 2019   P-value (F-stat)                0.0914
Time:                        13:42:23   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog           0.4411     0.4451     0.9911     0.3216     -0.4312      1.3134
educ           0.0592     0.0351     1.6878     0.09

In [4]:
from statsmodels.api import add_constant

In [28]:
import numpy as np
data = df1.dropna()
data = add_constant(data, has_constant='add')

In [29]:
res_second = IV2SLS(np.log(data.wage), data[['const']], data.educ, data.fatheduc).fit(cov_type='unadjusted')
print(res_second)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   wage   R-squared:                      0.0934
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0913
No. Observations:                 428   F-statistic:                    2.8487
Date:                Sun, Oct 13 2019   P-value (F-stat)                0.0914
Time:                        11:39:07   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.4411     0.4451     0.9911     0.3216     -0.4312      1.3134
educ           0.0592     0.0351     1.6878     0.09

In [ ]:
hacer manual, primero lka reducida y luego la estructural

## Ejemplo 15.2 Estimación del rendimiento de la educación para los hombres (wage2.dta)

In [2]:
dfej15_2 = pd.read_stata('wage2.dta')
dfej15_2.head()

,wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage
0,769,40,93,35,12,11,2,31,1,0,0,1,1,2.0,8.0,8.0,6.645091
1,808,50,119,41,18,11,16,37,1,0,0,1,1,NaN,14.0,14.0,6.694562
2,825,40,108,46,14,11,9,33,1,0,0,1,1,2.0,14.0,14.0,6.715384
3,650,40,96,32,12,13,7,32,1,0,0,1,4,3.0,12.0,12.0,6.476973
4,562,40,74,27,11,14,5,34,1,0,0,1,10,6.0,6.0,11.0,6.331502


In [4]:
regdfej15_2 = smf.ols('educ ~  sibs ', data = dfej15_2).fit()
print(regdfej15_2.summary())

                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     56.67
Date:                Fri, 17 Jul 2020   Prob (F-statistic):           1.22e-13
Time:                        12:14:29   Log-Likelihood:                -2034.4
No. Observations:                 935   AIC:                             4073.
Df Residuals:                     933   BIC:                             4083.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.1388      0.113    124.969      0.0

## Ejemplo 15.3 Estimación del efecto de fumaf sobre el peso de los recién nacidos(bwght.dta)

In [5]:
dfej15_3 = pd.read_stata('bwght.dta')
dfej15_3.head()

,faminc,cigtax,cigprice,bwght,fatheduc,motheduc,parity,male,white,cigs,lbwght,bwghtlbs,packs,lfaminc
0,13.5,16.5,122.300003,109,12.0,12.0,1,1,1,0,4.691348,6.8125,0.0,2.602690
1,7.5,16.5,122.300003,133,6.0,12.0,2,1,0,0,4.890349,8.3125,0.0,2.014903
2,0.5,16.5,122.300003,129,NaN,12.0,2,0,0,0,4.859812,8.0625,0.0,-0.693147
3,15.5,16.5,122.300003,126,12.0,12.0,2,1,0,0,4.836282,7.8750,0.0,2.740840
4,27.5,16.5,122.300003,134,14.0,12.0,2,1,1,0,4.897840,8.3750,0.0,3.314186


In [6]:
regdfej15_3 = smf.ols('packs ~  cigprice ', data = dfej15_3).fit()
print(regdfej15_3.summary())

                            OLS Regression Results                            
Dep. Variable:                  packs   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.1305
Date:                Fri, 17 Jul 2020   Prob (F-statistic):              0.718
Time:                        12:17:50   Log-Likelihood:                -291.47
No. Observations:                1388   AIC:                             586.9
Df Residuals:                    1386   BIC:                             597.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0674      0.103      0.658      0.5

## Ejemplo 15.4 Utilización de la proxlmidad a la universidad como una variable instrumental para la educación (card.dta)

In [5]:
data1 = pd.read_stata('card.dta')
data1.head(3)

,id,nearc2,nearc4,educ,age,fatheduc,motheduc,weight,momdad14,sinmom14,...,smsa66,wage,enroll,KWW,IQ,married,libcrd14,exper,lwage,expersq
0,2,0,0,7,29,NaN,NaN,158413.0,1,0,...,1,548,0,15.0,NaN,1.0,0.0,16,6.306275,256
1,3,0,0,12,27,8.0,8.0,380166.0,1,0,...,1,481,0,35.0,93.0,1.0,1.0,9,6.175867,81
2,4,0,0,12,34,14.0,12.0,367470.0,1,0,...,1,721,0,42.0,103.0,1.0,1.0,16,6.580639,256


In [6]:
from statsmodels.api import add_constant
data1 = add_constant(data1)
data1.head(3)

,const,id,nearc2,nearc4,educ,age,fatheduc,motheduc,weight,momdad14,...,smsa66,wage,enroll,KWW,IQ,married,libcrd14,exper,lwage,expersq
0,1.0,2,0,0,7,29,NaN,NaN,158413.0,1,...,1,548,0,15.0,NaN,1.0,0.0,16,6.306275,256
1,1.0,3,0,0,12,27,8.0,8.0,380166.0,1,...,1,481,0,35.0,93.0,1.0,1.0,9,6.175867,81
2,1.0,4,0,0,12,34,14.0,12.0,367470.0,1,...,1,721,0,42.0,103.0,1.0,1.0,16,6.580639,256


In [7]:
dep = ['wage']
endog = ['educ']
exog = ['const','exper','expersq','black','smsa','south','smsa66','reg662','reg663','reg664','reg665','reg666','reg667','reg668','reg669']
instr = ['nearc4']
data1 = data1[dep +exog+endog+ instr].dropna()

In [8]:
data1.head(3)

,wage,const,exper,expersq,black,smsa,south,smsa66,reg662,reg663,reg664,reg665,reg666,reg667,reg668,reg669,educ,nearc4
0,548,1.0,16,256,1,1,0,1,0,0,0,0,0,0,0,0,7,0
1,481,1.0,9,81,0,1,0,1,0,0,0,0,0,0,0,0,12,0
2,721,1.0,16,256,0,1,0,1,0,0,0,0,0,0,0,0,12,0


In [9]:
data1[exog]

,const,exper,expersq,black,smsa,south,smsa66,reg662,reg663,reg664,reg665,reg666,reg667,reg668,reg669
0,1.0,16,256,1,1,0,1,0,0,0,0,0,0,0,0
1,1.0,9,81,0,1,0,1,0,0,0,0,0,0,0,0
2,1.0,16,256,0,1,0,1,0,0,0,0,0,0,0,0
3,1.0,10,100,0,1,0,1,1,0,0,0,0,0,0,0
4,1.0,16,256,0,1,0,1,1,0,0,0,0,0,0,0
5,1.0,8,64,0,1,0,1,1,0,0,0,0,0,0,0
6,1.0,9,81,0,1,0,1,1,0,0,0,0,0,0,0
7,1.0,9,81,0,1,0,1,1,0,0,0,0,0,0,0
8,1.0,10,100,0,1,0,1,1,0,0,0,0,0,0,0
9,1.0,11,121,0,1,0,1,1,0,0,0,0,0,0,0


In [10]:
data1[endog]

,educ
0,7
1,12
2,12
3,11
4,12
5,12
6,18
7,14
8,12
9,12


In [11]:
data1[instr]

,nearc4
0,0
1,0
2,0
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [12]:
reg6 = IV2SLS(np.log(data1.wage), data1[exog], data1[endog], data1[instr]).fit()
print(reg6)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   wage   R-squared:                      0.2382
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2343
No. Observations:                3010   F-statistic:                    840.83
Date:                Sun, Oct 13 2019   P-value (F-stat)                0.0000
Time:                        16:17:01   Distribution:                 chi2(15)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.6662     0.9085     4.0352     0.0001      1.8855      5.4468
exper          0.1083     0.0233     4.6376     0.00

In [ ]:
# Debe cuidarse por que de statsmodelos también se importa IV2SLS . No pueden tener el mismo nombre y estar activos. 
# Debe sacarse uno de la memoria y activar el otro , o algo parecido

## Ejemplo 15.5 Rendimiento de la educación para las mujeres trabajadoras (mroz.dta)

In [8]:
dfej15_5 = pd.read_stata('mroz.dta')
dfej15_5.head()

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310.0,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800.0,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040.0,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300.0,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300.0,0.6215,12,14,9.5,1,7,20.100058,1.524272,49


In [9]:
regdfej15_5 = smf.ols('educ ~ exper + expersq + fatheduc + motheduc', data = dfej15_5).fit()
print(regdfej15_5.summary())


                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     66.52
Date:                Fri, 17 Jul 2020   Prob (F-statistic):           3.67e-48
Time:                        12:34:46   Log-Likelihood:                -1574.1
No. Observations:                 753   AIC:                             3158.
Df Residuals:                     748   BIC:                             3181.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.3667      0.267     31.370      0.0

In [14]:
regdfej15_5a = IV2SLS.from_formula('lwage ~ 1 + [educ~fatheduc + motheduc] + exper + expersq', data=dfej15_5).fit()
print(regdfej15_5a)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1357
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1296
No. Observations:                 428   F-statistic:                    18.611
Date:                Fri, Jul 17 2020   P-value (F-stat)                0.0003
Time:                        12:43:48   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0481     0.4278     0.1124     0.9105     -0.7903      0.8865
exper          0.0442     0.0155     2.8546     0.00

c:\python\lib\site-packages\linearmodels\utility.py:502: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## Ejemplo 15.6 Utilización de dos calificaciones de examen como indicadones de la habilidad* (wage2.dta)

In [15]:
print(IV2SLS.from_formula('lwage ~ 1+ educ + exper + tenure + married + south + urban + black + [IQ ~ KWW]', data = dfej15_2).fit())

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1900
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1830
No. Observations:                 935   F-statistic:                    356.33
Date:                Fri, Jul 17 2020   P-value (F-stat)                0.0000
Time:                        12:49:16   Distribution:                  chi2(8)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      4.5925     0.3501     13.117     0.0000      3.9063      5.2786
educ           0.0250     0.0187     1.3410     0.17

## Ejemplo 15.7 Rendimiento de la educación para las mujeres trabajadoras* (mroz.dta)

In [16]:
dfej15_51 = dfej15_5[(dfej15_5['inlf']==1)]
v2 = smf.ols('educ ~ exper + expersq + fatheduc + motheduc', data=dfej15_51).fit().resid
print(IV2SLS.from_formula('lwage ~1 + [educ ~ fatheduc + motheduc] + exper + expersq + v2 ', data = dfej15_51).fit())

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1624
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1544
No. Observations:                 428   F-statistic:                    87.093
Date:                Fri, Jul 17 2020   P-value (F-stat)                0.0000
Time:                        13:01:46   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0481     0.4196     0.1146     0.9087     -0.7744      0.8706
exper          0.0442     0.0150     2.9382     0.00

In [20]:
regdfej15_7 = smf.ols('lwage ~ educ + exper + expersq', data=dfej15_5).fit()
print(regdfej15_7.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     26.29
Date:                Fri, 17 Jul 2020   Prob (F-statistic):           1.30e-15
Time:                        13:06:24   Log-Likelihood:                -431.60
No. Observations:                 428   AIC:                             871.2
Df Residuals:                     424   BIC:                             887.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.5220      0.199     -2.628      0.0

## Ejemplo 15.8 Rendimiento de la educación para las mujeres trabajadoras* (mroz.dta)

In [24]:
dfej15_1 = dfej15_1.dropna()
u1 = (IV2SLS.from_formula('lwage ~1 + [educ ~ fatheduc + motheduc] + exper + expersq ', data = dfej15_1).fit()).resids
regdfej15_8 = smf.ols('u1 ~ exper + expersq + fatheduc + motheduc', data=dfej15_1).fit()
print(regdfej15_8.summary())

                            OLS Regression Results                            
Dep. Variable:                     u1   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                   0.09350
Date:                Fri, 17 Jul 2020   Prob (F-statistic):              0.984
Time:                        13:11:56   Log-Likelihood:                -436.70
No. Observations:                 428   AIC:                             883.4
Df Residuals:                     423   BIC:                             903.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0110      0.141      0.078      0.9

## Ejemplo 15.9 Efecto de la educación sobre la fertilidad (fertil1.dta)

In [25]:
dfej15_9 = pd.read_stata('fertil1.dta')
dfej15_9.head()

,year,educ,meduc,feduc,age,kids,black,east,northcen,west,...,y80,y82,y84,agesq,y74educ,y76educ,y78educ,y80educ,y82educ,y84educ
0,72,12,8,8,48,4,0,0,1,0,...,0,0,0,2304,0,0,0,0,0,0
1,72,17,8,18,46,3,0,0,0,0,...,0,0,0,2116,0,0,0,0,0,0
2,72,12,7,8,53,2,0,0,1,0,...,0,0,0,2809,0,0,0,0,0,0
3,72,12,12,10,42,2,0,0,1,0,...,0,0,0,1764,0,0,0,0,0,0
4,72,12,3,8,51,2,0,0,0,0,...,0,0,0,2601,0,0,0,0,0,0


In [26]:
print(IV2SLS.from_formula('kids ~ 1 + [educ ~ meduc + feduc] + age + agesq + black + east + northcen + west + farm + othrural + town + smcity + y74 + y76 + y78 + y80 + y82 + y84', data=dfej15_9).fit())
print(smf.ols('kids ~ educ + age + agesq + black + east + northcen + west + farm + othrural + town + smcity + y74 + y76 + y78 + y80 + y82 + y84', data=dfej15_9).fit().summary())

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   kids   R-squared:                      0.1281
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1148
No. Observations:                1129   F-statistic:                    150.13
Date:                Fri, Jul 17 2020   P-value (F-stat)                0.0000
Time:                        13:13:45   Distribution:                 chi2(17)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -7.2412     3.1890    -2.2707     0.0232     -13.491     -0.9910
age            0.5236     0.1395     3.7540     0.00

## Ejemplo 15.10 Formación laboral y productividad de los trabajadores (jtrain.dta)

In [28]:
dfej15_10 = pd.read_stata('jtrain.dta')
dfej15_10.head()

,year,fcode,employ,sales,avgsal,scrap,rework,tothrs,union,grant,...,grant_1,clscrap,cgrant,clemploy,clsales,lavgsal,clavgsal,cgrant_1,chrsemp,clhrsemp
0,1987,410032.0,100.0,47000000.0,35000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,10.463103,NaN,NaN,NaN,NaN
1,1988,410032.0,131.0,43000000.0,37000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,0.270027,-0.088949,10.518673,0.055570,0.0,-8.946565,-1.165385
2,1989,410032.0,123.0,49000000.0,39000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,-0.063013,0.130621,10.571317,0.052644,0.0,0.198597,0.047832
3,1987,410440.0,12.0,1560000.0,10500.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,9.259130,NaN,NaN,NaN,NaN
4,1988,410440.0,13.0,1970000.0,11000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,0.080043,0.233347,9.305651,0.046520,0.0,0.000000,0.000000


In [29]:
dfej15_10 = dfej15_10[(dfej15_10['year']==1988)]
print(smf.ols(formula='chrsemp ~ cgrant + 1', data=dfej15_10).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                chrsemp   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.387
Method:                 Least Squares   F-statistic:                     79.37
Date:                Fri, 17 Jul 2020   Prob (F-statistic):           5.69e-15
Time:                        13:14:45   Log-Likelihood:                -515.77
No. Observations:                 125   AIC:                             1036.
Df Residuals:                     123   BIC:                             1041.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5093      1.558      0.327      0.7